# Step 2: Finding Relevant Data

Add your API key to the cell below then run it.

In [9]:
import os
if 'A306709' in os.environ['USERNAME']:
    print("Running on Christophs computer: update proxy settings.")
    os.environ["http_proxy"] = "http://sia-lb.telekom.de:8080"
    os.environ["https_proxy"] = "http://sia-lb.telekom.de:8080"
else:
    print("Running on any computer but not Christophs: don't update any proxy settings.")

import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = os.getenv("OPENAI_API_KEY")


Running on Christophs computer: update proxy settings.


The code below loads in the embeddings you previously created. Run it as-is.

In [10]:
import numpy as np
import pandas as pd

df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df

,text,embeddings
0,– 2022 (MMXXII) was a common year starting on...,"[-2.4516581106581725e-05, -0.01798415742814541..."
1,– The year began with another wave in the COV...,"[-0.004297760780900717, -0.01981227844953537, ..."
2,– 2022 was also dominated by wars and armed c...,"[-0.009628579951822758, -0.015318196266889572,..."
3,– The ongoing Russian invasion of Ukraine esc...,"[-0.014748847112059593, -0.007639929186552763,..."
4,January 1 – The Regional Comprehensive Econom...,"[-0.0005859995144419372, -0.024158434942364693..."
...,...,...
193,December 24 – 2022 Fijian general election: Th...,"[-0.011724342592060566, -0.009384616278111935,..."
194,December 29 – Brazilian football legend Pelé d...,"[-0.007581493351608515, 0.0041535887867212296,..."
195,December 31 – Former Pope Benedict XVI dies at...,"[0.023601267486810684, 0.007735467050224543, -..."
196,December 7 – The world population was estimate...,"[-0.004104986786842346, -0.014414746314287186,..."


## TODO 1: Create Embeddings for the User's Question

In the previous exercise, you were given the code to create embeddings for everything in the dataset. Now your task is to create embeddings for just one string: the user's question. Assign the result to the variable `question_embeddings`. This variable should contain a list of 1,536 floating point numbers, and the provided code will print the first 100 once `question_embeddings` has been created correctly.

If you're getting stuck, you can click to reveal the solution then copy and paste this into the cell below.

---

<details>
    <summary style="cursor: pointer"><strong>Solution (click to show/hide)</strong></summary>

```python
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
USER_QUESTION = """What were the estimated damages of the 2023 \
Turkey-Syria earthquake?"""

# Generate the embedding response
response = openai.Embedding.create(
    input=USER_QUESTION,
    engine=EMBEDDING_MODEL_NAME
)

# Extract the embeddings from the response
question_embeddings = response["data"][0]["embedding"]

print(question_embeddings[:100])
```

</details>

In [11]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
USER_QUESTION = """What were the estimated damages of the 2023 \
Turkey-Syria earthquake?"""

# Generate the embedding response
response = openai.Embedding.create(
    input=USER_QUESTION,
    engine=EMBEDDING_MODEL_NAME
)

# Extract the embeddings from the response
question_embeddings = response["data"][0]["embedding"]

print(question_embeddings[:100])


# meine Methode, gleiches Ergebnis!:
#question_embeddings2 = get_embedding(USER_QUESTION, engine=EMBEDDING_MODEL_NAME)
#print(question_embeddings2[:10])

[0.0055134412832558155, -0.024900270625948906, 0.0023327376693487167, -0.012057743035256863, -0.02148180454969406, 0.0025389099027961493, -0.03381222486495972, -0.013101905584335327, 0.002271218691021204, -0.015004009008407593, 0.0162410419434309, 0.044320352375507355, -0.010262050665915012, -0.013308077119290829, 0.015070516616106033, -0.005400379188358784, 0.012456785887479782, -0.013135158456861973, 0.008339994587004185, -0.0057362401857972145, -0.0074754017405211926, 0.011472480371594429, 0.012396929785609245, -0.009018367156386375, 0.015123722143471241, 0.032162848860025406, 0.008027411065995693, -0.0023377256002277136, -0.0007943445816636086, -0.012204058468341827, 0.005523417145013809, -0.006550952792167664, -0.023450415581464767, 0.015256736427545547, -0.03240227326750755, -0.007575162220746279, 0.0068834880366921425, -0.0048450445756316185, 0.023344002664089203, -0.007927650585770607, 0.010075830854475498, 0.028624670580029488, 0.01296889130026102, -0.007708176504820585, -0.01

## TODO 2: Find Cosine Distances

Create a new list called `distances`, which represents the cosine distances between `question_embeddings` and each value in the `'embeddings'` column of `df`.

If you're getting stuck, you can click to reveal the solution then copy and paste this into the cell below.

---

<details>
    <summary style="cursor: pointer"><strong>Solution (click to show/hide)</strong></summary>

```python
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = distances_from_embeddings(
    question_embeddings,
    df["embeddings"],
    distance_metric="cosine"
)

print(distances[:100])
```

</details>

In [12]:
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = distances_from_embeddings(question_embeddings, df["embeddings"].tolist(), distance_metric="cosine")

print(distances[:100])

[np.float64(0.2820175817769158), np.float64(0.24720548525075703), np.float64(0.21704039447578993), np.float64(0.2754164902250461), np.float64(0.2722767751288506), np.float64(0.2653465679857081), np.float64(0.27707150819742954), np.float64(0.24430287923902738), np.float64(0.26126383768436245), np.float64(0.25353599110736724), np.float64(0.29493572636323073), np.float64(0.3071626272479342), np.float64(0.26303567417295914), np.float64(0.23019449279656168), np.float64(0.29961079242700783), np.float64(0.2409053976477057), np.float64(0.23255757786855114), np.float64(0.27062891506523024), np.float64(0.30448700064088297), np.float64(0.2613078462561649), np.float64(0.24824491008417016), np.float64(0.2673761487155515), np.float64(0.30140726636278947), np.float64(0.2262194221194651), np.float64(0.306428923601884), np.float64(0.25162918374990173), np.float64(0.26716229044103945), np.float64(0.2593078691804219), np.float64(0.24951788602432157), np.float64(0.26113144045631653), np.float64(0.27018155

## Sorting by Distance

The code below uses the `distances` list to update `df` then sorts `df` to find the most related rows. Shorter distance means more similarity, so we'll use an ascending sorting order. Run the cell below as-is.

In [13]:
df["distances"] = distances
df.sort_values(by="distances", ascending=True, inplace=True)
df.head(5)

,text,embeddings,distances
100,June 22 – A 6.2 earthquake strikes the Durand ...,"[-0.0016951377037912607, -0.002599864499643445...",0.178249
183,November 21 – A 5.6 earthquake strikes near Ci...,"[0.0040461234748363495, -0.004114207345992327,...",0.180490
119,July 27 – A 7.0 earthquake strikes the island ...,"[0.002082424471154809, 0.00398823618888855, 0....",0.189535
136,September 5 – A 6.8 earthquake strikes Luding ...,"[0.012638013809919357, 0.016330983489751816, 0...",0.201293
96,June 8 – 2022 South Khorasan train derailment:...,"[-0.019589906558394432, -0.02260781079530716, ...",0.209306


Great, now we have the dataset sorted from most relevant to least relevant! Let's save this as a CSV so we can load it in the next step and compose a custom prompt.

Run the cell below as-is.

In [14]:
df.to_csv("distances-earthquake.csv")